In [2]:
import pandas as pd
import numpy as np

# ========================================
# STEP 1: LOAD THE TRANSFORMED DATASETS
# ========================================
print("Loading transformed datasets...")

students = pd.read_csv('outputs/silver_students_transformed.csv')
courses = pd.read_csv('outputs/silver_courses_transformed.csv')
assessments = pd.read_csv('outputs/silver_assessments_transformed.csv')

print(f"✓ Students: {len(students)} records")
print(f"✓ Courses: {len(courses)} records")
print(f"✓ Assessments: {len(assessments)} records")

# ========================================
# STEP 2: PRE-MERGE PREPARATION
# ========================================
print("\nPreparing data for merge...")

# Standardize course codes BEFORE merging (important!)
courses['Course_Code'] = courses['Course_Code'].str.replace('-', '').str.replace(' ', '').str.upper().str.strip()
assessments['Course_Code'] = assessments['Course_Code'].str.replace('-', '').str.replace(' ', '').str.upper().str.strip()

print("✓ Course codes standardized")

# ========================================
# STEP 3: MERGE DATASETS
# ========================================
print("\nMerging datasets...")

# Step 1: Merge Students with Assessments (on Student_ID)
integrated = students.merge(
    assessments,
    on='Student_ID',
    how='left',  # Keep all students even if no assessments
    suffixes=('_student', '_assessment')
)

print(f"✓ After Students + Assessments: {len(integrated)} records")

# Step 2: Merge with Courses (on Course_Code and Campus_ID)
# Need to handle the Campus_ID suffix from previous merge
if 'Campus_ID_student' in integrated.columns:
    # Rename for merge
    integrated = integrated.rename(columns={'Campus_ID_student': 'Campus_ID'})
    
integrated = integrated.merge(
    courses,
    on=['Course_Code', 'Campus_ID'],
    how='left',  # Keep all records even if course not found
    suffixes=('', '_course')
)

print(f"✓ After adding Courses: {len(integrated)} records")

# ========================================
# STEP 4: CONFLICT RESOLUTION
# ========================================
print("\nResolving conflicts...")

# Handle name conflicts (same ID, different spelling)
if 'Full_Name' in integrated.columns:
    name_conflicts = integrated.groupby('Student_ID')['Full_Name'].nunique()
    conflicts = name_conflicts[name_conflicts > 1]
    
    if len(conflicts) > 0:
        print(f"Found {len(conflicts)} students with name variations")
        
        for student_id in conflicts.index:
            most_common_name = integrated[integrated['Student_ID'] == student_id]['Full_Name'].mode()[0]
            integrated.loc[integrated['Student_ID'] == student_id, 'Full_Name'] = most_common_name
        
        print(f"✓ Resolved {len(conflicts)} name conflicts")
    else:
        print("✓ No name conflicts found")

# Handle duplicate column suffixes
# Keep the primary value, drop the suffix columns
duplicate_cols = [col for col in integrated.columns if col.endswith(('_student', '_assessment', '_course'))]

for col in duplicate_cols:
    base_col = col.rsplit('_', 1)[0]
    if base_col in integrated.columns:
        # Fill missing values from the suffixed column
        integrated[base_col] = integrated[base_col].fillna(integrated[col])
        integrated = integrated.drop(columns=[col])

print("✓ Resolved duplicate columns")

# ========================================
# STEP 5: SAVE INTEGRATED DATASET
# ========================================
print("\nSaving integrated dataset...")

integrated.to_csv('outputs/gold_integrated.csv', index=False)

print(f"\n✓ Saved: outputs/gold_integrated.csv")
print(f"  Records: {len(integrated):,}")
print(f"  Columns: {len(integrated.columns)}")

# Display sample
print("\nSample of integrated data:")
print(integrated.head())

Loading transformed datasets...
✓ Students: 589 records
✓ Courses: 75 records
✓ Assessments: 5905 records

Preparing data for merge...
✓ Course codes standardized

Merging datasets...
✓ After Students + Assessments: 5905 records
✓ After adding Courses: 5905 records

Resolving conflicts...
✓ No name conflicts found
✓ Resolved duplicate columns

Saving integrated dataset...

✓ Saved: outputs/gold_integrated.csv
  Records: 5,905
  Columns: 42

Sample of integrated data:
  Student_ID First_Name Last_Name         DOB         Phone  \
0    RPH0001      Grace   Uwimana  2000-12-04  2.507993e+11   
1    RPH0001      Grace   Uwimana  2000-12-04  2.507993e+11   
2    RPH0001      Grace   Uwimana  2000-12-04  2.507993e+11   
3    RPH0001      Grace   Uwimana  2000-12-04  2.507993e+11   
4    RPH0001      Grace   Uwimana  2000-12-04  2.507993e+11   

                    Email  Level  Intake_Year  Campus_ID      Full_Name  ...  \
0  grace.uwimana@rp.ac.rw      1         2023          1  Grace Uwima